In [2]:
#import sys
#sys.path.append('C:/Users/carva/CDPRO/PROJETOS/AGRONEGÓCIO')
from arquivo_preprocessado import preprocessing
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
import warnings
warnings.filterwarnings ("ignore")


In [3]:
X_train , X_test , y_train, y_test = preprocessing()

In [4]:
X_train

,Estimated_Insects_Count,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Crop_Type_0,Crop_Type_1,Soil_Type_0,Soil_Type_1,Pesticide_Use_Category_1,Pesticide_Use_Category_2,Pesticide_Use_Category_3,Season_1,Season_2,Season_3
68355,625,5,29.0,2,False,True,True,False,False,True,False,True,False,False
79353,342,20,27.0,4,True,False,True,False,False,True,False,True,False,False
70521,2016,20,35.0,7,True,False,False,True,False,True,False,True,False,False
34921,1786,60,17.0,29,True,False,False,True,False,True,False,False,True,False
49121,531,30,15.0,13,True,False,True,False,False,True,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59502,1478,20,45.0,7,True,False,True,False,False,True,False,False,False,True
11907,1212,15,25.0,18,True,False,False,True,False,True,False,True,False,False
55312,1678,20,16.0,24,True,False,False,True,False,True,False,False,False,True
65138,789,40,24.0,14,True,False,False,True,False,True,False,False,True,False
